
# Fastest BFS search algorithm implementation applied to 9-Puzzle

Unsolvable pair of starting and target boards are exhausted in 0.5 seconds (180k steps).

For reference, a Python implementation takes around 180 seconds, e.g.:

* `https://github.com/g-quest/artificial_intelligence_9_puzzle`

    * Run: `python driver.py bfs 5,1,2,3,4,7,6,8,0`
    
Thus, this Julia implementation is **360 times faster**.

In [24]:

using BenchmarkTools
using StaticArrays
import Base.display
import Random
N = 3
M = 3

function get_problem()
    init_state = Random.shuffle(UInt8.(0 : N * M - 1))
    goal_state = Random.shuffle(UInt8.(0 : N * M - 1))
    display(init_state)
    display(goal_state)
    return init_state, goal_state
end
    

display(x::Union{Vector{UInt8}, SVector{9, UInt8}}) = display(reshape(Int64.(x), (3, 3)))


display (generic function with 31 methods)

In [25]:
st = SVector{4, SVector{9, UInt8}}(SVector{9, UInt8}(1:9) for i in 1:4)  # state copied 4 times.

# generating possible next moves given current state
# in particular, which locations can "0" be swapped with
MOVES_ = Int64.(zeros(4, 9))  # 4 max possible moves for 9 possible states
for i in 1:9  # 9 possible locations for "0"
    a = collect(1:9)
    a[i] = 0  # creative a vector with 0 in that location.
    aa = reshape(a, (3, 3))
    ii = findall(x->x==0, aa)[1]  # location of "0" but inside the matrix not the vector.
    
    if ii[1] < 3  # can be increased ==> MOVE zero down
        tmp = reshape(collect(1:9), (3, 3))
        tmp[ii[1] + 1, ii[2]] = 0
        ii_ = findall(x->x==0, reshape(tmp, 9))[1]
        MOVES_[1, i] = ii_
    end
    if ii[1] > 1  # can be decreased ==> MOVE zero up
        tmp = reshape(collect(1:9), (3, 3))
        tmp[ii[1] - 1, ii[2]] = 0
        ii_ = findall(x->x==0, reshape(tmp, 9))[1]
        MOVES_[2, i] = ii_    
    end  
    if ii[2] < 3  # can  be increased ==> MOVE zero left
        tmp = reshape(collect(1:9), (3, 3))
        tmp[ii[1], ii[2] + 1] = 0
        ii_ = findall(x->x==0, reshape(tmp, 9))[1]
        MOVES_[3, i] = ii_                
    end
    if ii[2] > 1  # can be increased ==> MOVE zero right
        tmp = reshape(collect(1:9), (3, 3))
        tmp[ii[1], ii[2] - 1] = 0
        ii_ = findall(x->x==0, reshape(tmp, 9))[1]
        MOVES_[4, i] = ii_    
    end
end

const MOVES = SVector([SVector(x...) for x in [[2, 0, 4, 0], [3, 1, 5, 0], [0, 2, 6, 0], [5, 0, 7, 1],
                [6, 4, 8, 2], [0, 5, 9, 3],
                [8, 0, 0, 4], [9, 7, 0 ,5], [0, 8, 0, 6]]]...);  
# TODO: Make UInt8

display(MOVES_)
display(MOVES)

4×9 Matrix{Int64}:
 2  3  0  5  6  0  8  9  0
 0  1  2  0  4  5  0  7  8
 4  5  6  7  8  9  0  0  0
 0  0  0  1  2  3  4  5  6

9-element SVector{9, SVector{4, Int64}} with indices SOneTo(9):
 [2, 0, 4, 0]
 [3, 1, 5, 0]
 [0, 2, 6, 0]
 [5, 0, 7, 1]
 [6, 4, 8, 2]
 [0, 5, 9, 3]
 [8, 0, 0, 4]
 [9, 7, 0, 5]
 [0, 8, 0, 6]

In [26]:
function get_blank_index1(state)
    return findall(state .== 0)[1]
end

@inline function get_blank_index2(state)
    for (idx, item) in enumerate(state)
        if item == 0
            return idx
        end
    end
    println("Failed to find index of zero tile.")
    return 0 # avoid type instability in the return, as indicated by @code_warntype
end

state = SVector(UInt8.([1,2,3,8,6,4,7,5,0])...)
@btime get_blank_index1(state)
@btime get_blank_index2(state)


  53.049 ns (1 allocation: 96 bytes)
  21.407 ns (0 allocations: 0 bytes)


9

In [27]:
"""The object lives in memory and has reference to its parent
"""
mutable struct ReferenceVertex
   state::Vector{UInt8}
   parent::ReferenceVertex
   ReferenceVertex(state, parent) = new(state, parent)
   ReferenceVertex(state) = begin
        x = new(state)
        x.parent = x  # MUTATION
    end
end


@inline function get_next_states_only(st, state)
    idx = get_blank_index2(state)  # location of zero
    counter = 0
    for item in MOVES[idx]
        if item != 0
            tmp = state[item]
            nxt = setindex(setindex(state, 0, item), tmp, idx)
            counter += 1
            st = setindex(st, nxt, counter)
        end
    end
    return st, counter
end


get_next_states_only (generic function with 1 method)

In [28]:

function bfs_with_reference_objects(init_state, goal_state; verbose=true)
    
    EXPLORED_H = Set{UInt64}()  # container of hashes for fast search.    
    EXPLORED = Vector{ReferenceVertex}()  # container of Frontier objects.

    FRONTIER = Vector{ReferenceVertex}()  # container of Frontier objects.
    FRONTIER_H = Set{UInt64}()  # container of hashes.
    
    goal_state = SVector(goal_state...)
    push!(FRONTIER, ReferenceVertex(init_state))
    counter = 0
    
    while length(FRONTIER) > 0
        counter += 1
        obj = popfirst!(FRONTIER)
        current_state = SVector(obj.state...)
        
        if current_state == goal_state
            if verbose
                println("Solved")
                println("Frontier size = $(length(FRONTIER))")
                println("Explored size = $(length(EXPLORED_H))")
            end
            return obj  # trace_solution(current_state, front_pointer)
        else
            
            states, num = get_next_states_only(st, current_state)

            @inbounds for index ∈ 1:num
                item = states[index]
                if (hash(item) ∉ EXPLORED_H) & (hash(item) ∉ FRONTIER_H)
                    push!(FRONTIER, ReferenceVertex(item, obj))
                    push!(FRONTIER_H, hash(item))
                end
            end
            
            push!(EXPLORED_H, hash(current_state))
            push!(EXPLORED, obj)
            
        end  # if statement  
    end  # while loop
    
    if verbose
        println("No solution")
        println("Frontier size = $(length(FRONTIER))")
        println("Explored size = $(length(EXPLORED_H))")
    end
    return nothing
end


function trace_solution(goal_obj, print_it=true)
    if goal_obj == nothing
       return nothing 
    end
    result = []
    current_obj = goal_obj
    while current_obj != current_obj.parent
        push!(result, current_obj)
        current_obj = current_obj.parent
    end
    push!(result, current_obj)
    result = reverse(result)
    print("Solution length = ", length(result))
    
    if print_it  && length(result) < 50
       for step in result
           display(step.state)
       end 
    end
    return result
    
end


trace_solution (generic function with 2 methods)

In [29]:
init_state, goal_state = get_problem();

3×3 Matrix{Int64}:
 3  1  8
 6  2  4
 7  0  5

3×3 Matrix{Int64}:
 4  0  8
 7  3  2
 5  1  6

In [30]:
sol = bfs_with_reference_objects(init_state, goal_state; verbose=true);
trc = trace_solution(sol);

No solution
Frontier size = 0
Explored size = 181440


In [31]:
@btime bfs_with_reference_objects(init_state, goal_state; verbose=false);

  491.323 ms (4233708 allocations: 127.98 MiB)
